In [ ]:
# %% [markdown]
# # Trader Performance vs Market Sentiment Analysis
# **Data Science Intern Assignment - Primetrade.ai**

# %% [markdown]
# ## Part A: Data Loading & Preparation

# %%
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('..')
from src.data_loader import load_fear_greed_data, load_trader_data
from src.data_cleaner import clean_fear_greed, clean_trader_data
from src.feature_engineering import create_daily_metrics, merge_with_sentiment
from src.visualization import plot_pnl_by_sentiment, plot_behavior_changes, create_trader_segments

# %%
# Load data
fear_greed = load_fear_greed_data('../data/raw/fear_greed_index.csv')
trader_data = load_trader_data('../data/raw/hyperliquid_trades.csv')

# %%
# Clean data
fear_greed_clean = clean_fear_greed(fear_greed)
trader_clean = clean_trader_data(trader_data)

# %%
# Create daily metrics
daily_metrics = create_daily_metrics(trader_clean)
print(f"Daily metrics shape: {daily_metrics.shape}")
daily_metrics.head()

# %%
# Merge with sentiment
final_df = merge_with_sentiment(daily_metrics, fear_greed_clean)
print(f"Final dataset shape: {final_df.shape}")
final_df.head()

# %% [markdown]
# ## Part B: Analysis

# %% [markdown]
# ### Q1: Does performance differ between Fear vs Greed days?

# %%
# Summary statistics
performance_by_sentiment = final_df.groupby('Classification').agg({
    'total_pnl': ['mean', 'median', 'std'],
    'win_rate': 'mean',
    'max_drawdown': 'mean'
}).round(2)
print("Performance Metrics by Sentiment:")
print(performance_by_sentiment)

# %%
# Visualization
fig1 = plot_pnl_by_sentiment(final_df)
plt.savefig('../outputs/charts/pnl_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.show()

# %% [markdown]
# **Key Findings for Q1:**
# - Write 2-3 bullet points here based on actual data

# %% [markdown]
# ### Q2: Do traders change behavior based on sentiment?

# %%
behavior_by_sentiment = final_df.groupby('Classification').agg({
    'trade_count': 'mean',
    'avg_leverage': 'mean',
    'avg_trade_size': 'mean',
    'long_pct': 'mean' if 'long_pct' in final_df.columns else 'mean'
}).round(2)
print("Behavior Changes by Sentiment:")
print(behavior_by_sentiment)

# %%
fig2 = plot_behavior_changes(final_df)
plt.savefig('../outputs/charts/behavior_changes.png', dpi=300, bbox_inches='tight')
plt.show()

# %% [markdown]
# **Key Findings for Q2:**
# - Write 2-3 bullet points here

# %% [markdown]
# ### Q3: Identify 2-3 trader segments

# %%
trader_segments = create_trader_segments(final_df)
print("Trader Segments Summary:")
print(trader_segments.groupby('leverage_segment').size())
print("\n")
print(trader_segments.groupby('frequency_segment').size())
print("\n")
print(trader_segments.groupby('performance_segment').size())

# %%
# Segment performance analysis
segment_performance = final_df.merge(
    trader_segments[['account', 'leverage_segment', 'frequency_segment', 'performance_segment']], 
    on='account'
)

# Analyze how different segments perform on Fear vs Greed days
segment_analysis = segment_performance.groupby(
    ['leverage_segment', 'Classification']
)['total_pnl'].mean().unstack()
print("Segment Performance on Fear vs Greed:")
print(segment_analysis.round(2))

# %%
# Visualize segment differences
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, segment in enumerate(['leverage_segment', 'frequency_segment', 'performance_segment']):
    segment_data = segment_performance.groupby([segment, 'Classification'])['total_pnl'].mean().unstack()
    segment_data.plot(kind='bar', ax=axes[i])
    axes[i].set_title(f'PnL by {segment}')
    axes[i].set_ylabel('Average PnL')
    axes[i].legend(title='Sentiment')

plt.tight_layout()
plt.savefig('../outputs/charts/segment_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# %% [markdown]
# ### Q4: Provide at least 3 insights with evidence

# %% [markdown]
# **Insight 1: [Title]**
# - Evidence: [Show chart/table]
# - Explanation: [Why this matters]

# %%
# Example insight visualization
# [Your code here]

# %% [markdown]
# **Insight 2: [Title]**
# - Evidence: [Show chart/table]
# - Explanation: [Why this matters]

# %% [markdown]
# **Insight 3: [Title]**
# - Evidence: [Show chart/table]
# - Explanation: [Why this matters]

# %% [markdown]
# ## Part C: Actionable Output

# %% [markdown]
# ### Strategy Idea 1: Sentiment-Based Leverage Adjustment
# 
# **Rule of thumb:** During Fear days, traders in the high-leverage segment should reduce leverage by X%, as they historically lose Y% more on these days.
# 
# **Implementation:** [Brief description of how to implement]

# %% [markdown]
# ### Strategy Idea 2: Counter-Cyclical Trading for Specific Segments
# 
# **Rule of thumb:** Infrequent traders actually perform better during Fear days (show +X% PnL vs Greed days). These traders should increase activity during market fear.
# 
# **Implementation:** [Brief description]

# %% [markdown]
# ## Bonus: Optional Enhancements

# %% [markdown]
# ### Bonus 1: Predictive Model
# 
# Predict next-day profitability using:
# - Today's sentiment
# - Trader's recent behavior
# - Market conditions

# %%
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Create features and target
# [Code here]

# %% [markdown]
# ### Bonus 2: Clustering Analysis

# %%
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Cluster traders into behavioral archetypes
# [Code here]

# %% [markdown]
# ### Bonus 3: Dashboard (Streamlit)
# 
# See `../dashboard/app.py` for interactive exploration